In [7]:
import pandas as pd 
import numpy as np

df = pd.read_csv(r"C:\Users\shaif\OneDrive\Desktop\gamezone_orders_data.csv", sep=",", encoding="utf-8",keep_default_na=False, na_values=[""], dtype=str)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21864 entries, 0 to 21863
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   USER_ID                          21864 non-null  object
 1   ORDER_ID                         21864 non-null  object
 2   PURCHASE_TS                      21864 non-null  object
 3   PURCHASE_TS_CLEANED              21863 non-null  object
 4   PURCHASE_YEAR                    21863 non-null  object
 5   PURCHASE_MONTH                   21863 non-null  object
 6   TIME_TO_SHIP                     21863 non-null  object
 7   SHIP_TS                          21864 non-null  object
 8   PRODUCT_NAME                     21864 non-null  object
 9   PRODUCT_NAME_CLEANED             21864 non-null  object
 10  PRODUCT_ID                       21864 non-null  object
 11   USD_PRICE                       21859 non-null  object
 12  PURCHASE_PLATFORM               

In [ ]:
# data cleaning 

df.columns = [col.lower() for col in df.columns] # lowering all column names 

df.columns = df.columns.str.strip()  # removes spaces around names

df['purchase_ts_cleaned'] = pd.to_datetime(df['purchase_ts_cleaned'], errors='coerce', dayfirst=True) # converting to datetime

df['ship_ts'] = pd.to_datetime(df['ship_ts'], errors='coerce', dayfirst=True) #  Convert to datetime


df['revenue'] = df['revenue'].replace(r'[\$,]', '', regex=True)
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')

df['region'] = df['region'].astype(str).str.strip().replace('', pd.NA)
df['region'].isna().sum()

df['purchase_year'] = df['purchase_year'].fillna(0).astype(int)
df['purchase_month'] = df['purchase_month'].fillna(0).astype(int)
df['time_to_ship'] = df['time_to_ship'].fillna(0).astype(int)

df['country_code'] = df['country_code'].astype(str).str.strip().str.upper() #cleaning the countries 

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21864 entries, 0 to 21863
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   user_id                          21864 non-null  object        
 1   order_id                         21864 non-null  object        
 2   purchase_ts                      21864 non-null  object        
 3   purchase_ts_cleaned              21863 non-null  datetime64[ns]
 4   purchase_year                    21864 non-null  int32         
 5   purchase_month                   21864 non-null  int32         
 6   time_to_ship                     21864 non-null  int32         
 7   ship_ts                          21864 non-null  datetime64[ns]
 8   product_name                     21864 non-null  object        
 9   product_name_cleaned             21864 non-null  object        
 10  product_id                       21864 non-null  object   

In [24]:
# checking unique products 

df["product_name_cleaned"].value_counts()

product_name_cleaned
nintendo switch                   10386
27in 4k gaming monitor             4723
jbl quantum 100 gaming headset     4296
sony playstation 5 bundle           977
dell gaming mouse                   719
lenovo ideapad gaming 3             669
acer nitro v gaming laptop           87
razer pro gaming headset              7
Name: count, dtype: int64

In [105]:
df['product_name_cleaned'].nunique()


8

In [25]:
# unique countries 

df['country_code'].nunique()

152

In [26]:
# checking most countries 

df['country_code'].value_counts().head()

country_code
US    10294
GB     1808
CA      956
AU      893
DE      855
Name: count, dtype: int64

In [28]:
# marketing channels 

df['marketing_channel_cleaned'].value_counts()

marketing_channel_cleaned
direct          17434
email            3256
affiliate         721
social media      323
unknown           130
Name: count, dtype: int64

In [36]:
# purchase platform 

df['purchase_platform'].value_counts()

purchase_platform
website       19783
mobile app     2081
Name: count, dtype: int64

In [40]:
# account type

df['account_creation_method_cleaned'].value_counts()

account_creation_method_cleaned
desktop    16458
mobile      4232
unknown      826
tablet       323
tv            25
Name: count, dtype: int64

In [41]:
total_revenue = df['revenue'].sum()

total_revenue

6151266.49

In [42]:
total_orders = df['order_id'].count()
total_orders

21864

In [43]:
unique_customers = df['user_id'].nunique()
unique_customers

19851

In [46]:
average_order_value = total_revenue/total_orders

average_order_value.round(2)

281.34

In [59]:
monthly_summary = df.groupby('purchase_month').agg(
    total_revenue=('revenue', 'sum'),
    total_orders=('order_id', 'count'),
    unique_customers=('user_id', 'nunique')
).reset_index()

# Remove the invalid month (0)
monthly_summary = monthly_summary[monthly_summary['purchase_month'] != 0]

# Add AOV (Average Order Value)
monthly_summary['aov'] = (monthly_summary['total_revenue'] / monthly_summary['total_orders']).round(2)

# Sort months if needed
monthly_summary = monthly_summary.sort_values('purchase_month')

print(monthly_summary)


    purchase_month  total_revenue  total_orders  unique_customers     aov
1                1      499725.27          1821              1465  274.42
2                2      514735.06          1825              1818  282.05
3                3      417653.10          1509              1504  276.77
4                4      472963.91          1723              1712  274.50
5                5      468890.58          1727              1725  271.51
6                6      433190.52          1617              1596  267.90
7                7      448530.84          1693              1573  264.93
8                8      525416.75          1797              1654  292.39
9                9      618444.77          2177              1860  284.08
10              10      464867.06          1549              1520  300.11
11              11      556622.22          1955              1771  284.72
12              12      730204.45          2470              2063  295.63


In [60]:
monthly_summary.to_csv(r"E:\Projects\Gamezone Orders Data\monthly_stats.csv", index=False)


In [121]:
sales_kpis.to_csv(r"E:\Projects\Gamezone Orders Data\sales_kpis.csv", index=False)

In [137]:
product_summary.to_csv('product_summary.csv', index=False)
monthly_revenue_trend.to_csv('product_monthly_trend.csv', index=False)
product_channel_region.to_csv('product_channel_region.csv', index=False)